<H1>Testing inhibitory motifs</H1>

In [1]:
# loading python modules
import numpy as np
np.random.seed(0)
from matplotlib.pyplot import figure
from terminaltables import AsciiTable 

from __future__ import division

In [2]:
# loading custom inet modules
from inet import DataLoader
from inet.motifs import iicounter
from inet.utils import chem_squarematrix, elec_squarematrix

In [3]:
# use the dataset to create the null hypothesis
mydataset = DataLoader('../data/PV')

 161 syn  files loaded


<H2>Collect number of experiments </H2>

In [4]:
# e.g. mydataset.PV[2].values  will return the different configurations with 2 PV cells
nPV = range(9)
for i in nPV:
    nPV[i] = np.sum(mydataset.IN[i].values())
    nPV[i] = np.sum(mydataset.IN[i].values())

In [5]:
nPV

[0, 141, 18, 2, 0, 0, 0, 0, 0]

In [6]:
# for the moment, we only count experiments with 2 or 3 PVs
# later we use mydataset.PV[2:]
PV2 = sum(mydataset.IN[2].values())
PV3 = sum(mydataset.IN[3].values())
PV2, PV3

(18, 2)

<H2> Calculate empirical probabilities </H2>

In [8]:
PC = mydataset.motif.ii_chem_found/mydataset.motif.ii_chem_tested
PE = mydataset.motif.ii_elec_found/mydataset.motif.ii_elec_tested

PC2  = mydataset.motif.ii_c2_found/mydataset.motif.ii_c2_tested
Pdiv = mydataset.motif.ii_div_found/mydataset.motif.ii_div_tested

PC1E = mydataset.motif.ii_c1e_found/mydataset.motif.ii_c1e_tested
PC2E = mydataset.motif.ii_c2e_found/mydataset.motif.ii_c2e_tested



info = [
    ['key', 'Probability', 'Motif', 'Value'],
    ['ii_chem', 'P(C)', 'chemical synapse',PC ],
    ['ii_elec', 'P(E)', 'electrical synapse',PE ],
    ['','',''],
    ['ii_c2', 'P(C U C)','bidirectional chemical synapse',PC2],
    ['ii_div', '', 'divergent inhibitory motifs', Pdiv],
    ['ii_c1e', 'P(C U E)', 'electrical and unidirectional chemical', PC1E],
    ['ii_c2e', 'P(2C U E):','electrical and bidirectional chemical', PC2E],

]
print(AsciiTable(info).table)

+---------+-------------+----------------------------------------+----------------+
| key     | Probability | Motif                                  | Value          |
+---------+-------------+----------------------------------------+----------------+
| ii_chem | P(C)        | chemical synapse                       | 0.291666666667 |
| ii_elec | P(E)        | electrical synapse                     | 0.458333333333 |
|         |             |                                        |                |
| c2      | P(C U C)    | bidirectional chemical synapse         | 0.166666666667 |
| ii_div  |             | divergent inhibitory motifs            | 0.166666666667 |
| c1e     | P(C U E)    | electrical and unidirectional chemical | 0.145833333333 |
| c2e     | P(2C U E):  | electrical and bidirectional chemical  | 0.125          |
+---------+-------------+----------------------------------------+----------------+


<H2> Simulate random chemical synapses</H2>
from a random distribution whose probability is adjusted to the empirical probability found in the recordings.


In [ ]:
def mychem_simulation():
    """
    simulate inhibitory chemical connections of the dataset
    
    Return
    ------
    A iicounter object 
    """
    mycount = iicounter()
    for _ in range(PV2):
        mycount += iicounter(chem_squarematrix(size=2,prob = PC))

    for _ in range(PV3):
        mycount += iicounter(chem_squarematrix(size=3, prob = PC))
        
    return(mycount)

In [ ]:
print(mychem_simulation()) # one simulation, test the number of connection tested

In [ ]:
# must contain the same number of tested connections
for key in mychem_simulation().keys():
    print(key, mydataset.motif[key])

In [ ]:
# simulate the whole data set 10,000 times
n_chem = list()
n_bichem = list()
for _ in range(1000):
    syn_counter = mychem_simulation()
    n_chem.append(syn_counter['ii_chem']['found'])
    n_bichem.append(syn_counter['ii_c2']['found'])

If the null hypothesis is correctly implemented, we should see almost the same 
number of chemical synases as in the experiments.

In [ ]:
np.mean(n_chem) # on average 14 unidirectional connections


In [ ]:
mydataset.motif['ii_chem']['found']

If we found a number which is different form the empirical, we must revise our 
null hypothese.

In [ ]:
np.mean(n_bichem) # on average two

Define analiticaly the null hypothese:

In [ ]:
PC*PC*mydataset.motif['ii_c2']['tested'] # null hypothesis

In [ ]:
mydataset.motif['ii_c2']['found'] # however, we found more empirically

<H2> Simulate random electrical synapses</H2>
from a random distribution whose probability is adjusted to the empirical probability found in the recordings.

In [ ]:
def myelec_simulation():
    """
    simulate inhibitory electrical connections of the dataset
    
    Return
    ------
    A iicounter object 
    """
    mycount = iicounter()
    for _ in range(PV2):
        mycount += iicounter(elec_squarematrix(size=2,prob = PE))

    for _ in range(PV3):
        mycount += iicounter(elec_squarematrix(size=3, prob = PE))
        
    return(mycount)

In [ ]:
print(myelec_simulation()) # one simulation, test the number of connection tested

In [ ]:
# must contain the same number of tested connections
for key in myelec_simulation().keys():
    print(key, mydataset.motif[key])

In [ ]:
n_elec = list()
for _ in range(1000):
    syn_elec = myelec_simulation()
    n_elec.append(syn_elec['ii_elec']['found'])

Similarly, we must see almost the same 
number of electrical connections as with the experiments

In [ ]:
np.mean(n_elec)

In [ ]:
mydataset.motif.ii_elec_found # voila!

<H2>Simulate electrical and chemical synapses independently</H2>

In [ ]:
C = chem_squarematrix(size = 2, prob = PC)
E = elec_squarematrix(size = 2, prob = PE)
C + E # when a chemical (1) and electrical (2) synapse add , they have the motif 3

In [ ]:
def myii_simulation():
    """
    simulate inhibitory electrical and chemical connections of the dataset
    
    Return
    ------
    A iicounter object 
    """
    mycount = iicounter()
    for _ in range(PV2):
        C = chem_squarematrix(size = 2, prob = PC)
        E = elec_squarematrix(size = 2, prob = PE)
        
        S = C + E
        x, y = np.where(S==2) # test to eliminate '1' from the oposite direction
        mycoor = zip(y,x)
        for i,j in mycoor:
            if S[i,j]==1:
                S[i,j]=3
                S[j,i]=0
                
        mycount += iicounter( S ) 

    for _ in range(PV3):
        C = chem_squarematrix(size = 3, prob = PC)
        E = elec_squarematrix(size = 3, prob = PE)

        S = C + E
        x, y = np.where(S==2) # test to eliminate '1' from the oposite direction
        mycoor = zip(y,x)
        for i,j in mycoor:
            if S[i,j]==1:
                S[i,j]=3
                S[j,i]=0
                
        mycount += iicounter( S ) 

    return(mycount)

In [ ]:
myii_simulation()# one simulation, again, test the number of connections tested

In [ ]:
# must contain the same number of tested connections
for key in myii_simulation().keys():
    print(key, mydataset.motif[key])

In [ ]:
# simulate the whole data set 1,000 times
n_chem = list()
n_elec = list()
n_c1e = list()
n_c2e = list()
n_c2 = list()

for _ in range(1000):
    syn_counter = myii_simulation()
    n_chem.append( syn_counter['ii_chem']['found'] )
    n_elec.append( syn_counter['ii_elec']['found'] )
    n_c1e.append( syn_counter['ii_c1e']['found'] )
    n_c2e.append( syn_counter['ii_c2e']['found'] )
    n_c2.append( syn_counter['ii_c2']['found'])

In [ ]:
info = [
    ['Syn Motif', 'Simulations', 'Empirical'], 
    ['chemical', np.mean(n_chem),  mydataset.motif['ii_chem']['found']],
    ['electrical', np.mean(n_elec),  mydataset.motif['ii_elec']['found']],
    ['','',''],
    ['2 chem',np.mean(n_c2),mydataset.motif['ii_c2']['found']],
    ['1 chem + elec', np.mean(n_c1e),  mydataset.motif['ii_c1e']['found']],
    ['2 chem + elec', np.mean(n_c2e),  mydataset.motif['ii_c2e']['found']],
     ]
print(AsciiTable(info).table)

Let's see if the connections found correspond to the theoretical values for the complex motifs. 


<H3>c2: bidirectional chemical connections</H3>

In [ ]:
mydataset.motif['ii_c2']

In [ ]:
PC1 = mydataset.motif.ii_chem_found /mydataset.motif.ii_chem_tested
PC1

In [ ]:
# definition of the null hypothese is based on unidirectional connections
(PC1*PC1)*mydataset.motif.ii_c2_tested # this value must be close to the simulation

<H3>ce1: unidirectional chemical connections in the presence of an electrical synapse</H3>

In [ ]:
mydataset.motif['ii_c1e']

In [ ]:
PCE1 = mydataset.motif.ii_c1e_found /mydataset.motif.ii_c1e_tested
PCE1

In [ ]:
# definition of the null hypothese
(PC*PE)*mydataset.motif.ii_c1e_tested # this value must be close to the simulation

<H3>ce2: bidirectional chemical connections in the presence of an electrical synapse</H3>

In [ ]:
mydataset.motif['ii_c2e']

In [ ]:
PCE2 = mydataset.motif.ii_c2e_found /mydataset.motif.ii_c2e_tested
PCE2

In [ ]:
(PE*PC*PC)*mydataset.motif.ii_c2e_tested # this value must be close to the simulation

<H2>Calculating P-Values</H2>

In [ ]:
n_chem = np.array(n_chem)
n_elec = np.array(n_elec)
n_c1e = np.array(n_c1e)
n_c2e = np.array(n_c2e)
n_c2 = np.array(n_c2)


In [ ]:
pii_chem = len(n_chem[n_chem>mydataset.motif.ii_chem_found]) / n_chem.size
pii_elec = len(n_elec[n_elec>mydataset.motif.ii_elec_found])/ n_elec.size
pii_c1e = len(n_c1e[n_c1e>mydataset.motif.ii_c1e_found])/ n_c1e.size
pii_c2e = len(n_c2e[n_c2e>mydataset.motif.ii_c2e_found])/ n_c2e.size
pii_c2 = len(n_c2[n_c2e>mydataset.motif.ii_c2_found])/ n_c2.size

In [ ]:
info = [
    ['Syn Motif', 'Simulations', 'Empirical', 'P(Simulations)'], 
    ['chemical', np.mean(n_chem),  mydataset.motif.ii_chem_found, pii_chem],
    ['electrical', np.mean(n_elec),  mydataset.motif.ii_elec_found, pii_elec],
    ['', '', '', ''],
    ['2 chem bidirect', np.mean(n_c2),  mydataset.motif.ii_c2_found, pii_c2],
    ['1 chem + elec', np.mean(n_c1e),  mydataset.motif.ii_c1e_found, pii_c1e],
    ['2 chem + elec', np.mean(n_c2e),  mydataset.motif.ii_c2e_found, pii_c2e],
     ]
print(AsciiTable(info).table)

In [ ]:
from inet.plots import barplot

<H2> Plot chemical synapses alone</H2>

In [ ]:
fig = figure(1)
ax = fig.add_subplot(111)

ax = barplot(simulation = n_chem, n_found = mydataset.motif.ii_chem_found);
ax.set_title('Chemical synapses');
ax.set_ylim(ymax=20);

fig.savefig('ii_chem.pdf')

<H2>Plot electrical synapses alone </H2>

In [ ]:
fig = figure(2)
ax = fig.add_subplot(111)

ax = barplot(simulation = n_elec, n_found = mydataset.motif.ii_elec_found);
ax.set_title('Electrical synapses');
ax.set_ylim(ymax=20);

fig.savefig('ii_elec.pdf')

<H2>Plot bidirectional chemical synapses</H2>


In [ ]:
fig = figure(4)
ax = fig.add_subplot(111)

ax = barplot(simulation = n_c2, n_found = mydataset.motif.ii_c2_found);
ax.set_title('Bidirectional chemical');
ax.set_ylim(ymax=10);

fig.savefig('ii_c2.pdf')

<H2>Plot electrical and one chemical synapse alone </H2>

In [ ]:
fig = figure(4)
ax = fig.add_subplot(111)

ax = barplot(simulation = n_c1e, n_found = mydataset.motif.ii_c1e_found);
ax.set_title('Electrical and one chemical');
ax.set_ylim(ymax=10);

fig.savefig('ii_c1e.pdf')

<H2>Plot electrical and two chemical</H2>

In [ ]:
fig = figure(5)
ax = fig.add_subplot(111)

ax = barplot(simulation = n_c2e, n_found = mydataset.motif.ii_c2e_found);
ax.set_title('Electrical and two chemical');
ax.set_ylim(ymin  = 0, ymax=5);

fig.savefig('ii_c2d.pdf')